<span style="font-size: 14pt">MIPT, Advanced ML, Autumn 2017</span>

<span style="font-size: 16pt"> HW #9: Recommendation System

<span style="color:blue; font-size: 12pt">Andrey Saitgalin </span>
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">andrey.saitgalin@gmail.com</span>, 

<span style="color:blue; font-size: 12pt">Alexey Dral </span>,
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">aadral@gmail.com</span>

<h1 align="center">Organization Info</h1> 

**Дополнительный материал для выполнения дз**:
<a href= "https://github.com/ml-mipt/ml-mipt-part2/tree/master/2017/lectures">Лекции по курсу</a>

**Оформление дз**: 
- Присылайте выполненное задание на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall <номер_группы> <фамилия>``, к примеру -- ``ML2017_Aut_fall 401 ivanov``
- Выполненное дз сохраните в файл ``<фамилия>_<группа>_task<номер>.ipnb``, к примеру -- ``ivanov_491_task9.ipnb``

**Вопросы**:
- Присылайте вопросы на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall Question <Содержание вопроса>``

** Дедлайн**:
<span style="color:red; font-size: 12pt">16.12.2017 23:59 </span>

--------
- **PS1**: Мы используем автоматические фильтры, и просто не найдем ваше дз, если вы не аккуратно его подпишите.
- **PS2**: Напоминаем, что дедлайны жесткие, письма пришедшие после автоматически удаляются =( чтобы соблазна не было.

<h1 align="center">Check Questions (0%)</h1> 
Вопросы отсутствуют

<h1 align="center"> Practical tasks (100%)</h1>


# Построение content-based рекомендательной системы
## Part 1
Для каждого фильма получите описание в текстовом виде с IMDB или (и) The movie database (у них есть открытое API). https://www.themoviedb.org/?language=ru. Вывести описание фильма под id 778. Также можно использовать рецензии.

In [1]:
import tmdbsimple as tmdb
tmdb.API_KEY = 'b3cf4dab4c66584c6ee94f4ef5b0d390'

In [2]:
all_movies = dict()

In [3]:
from tqdm import tqdm_notebook

In [4]:
for i in range(10):
    try:
        movie = tmdb.Movies(i)
        all_movies[i] = movie.info()
        
        if i % 10 == 0:
            with open('movies_{}'.format(i), 'wb') as f:
                pickle.dump(all_movies, f)
    except:
        pass

In [7]:
import pickle

with open('movies_{}'.format(i), 'wb') as f:
    pickle.dump(all_movies, f)

In [132]:
import pandas as pd

all_movies = pd.read_csv('movies.csv')

In [134]:
all_movies.head()

,id,title,overview
0,426903,Buffalo Running,Individual photographs of the running of a buf...
1,421315,Human Figure in Motion,Muybridge first photographed the human figure ...
2,421316,Descending Stairs and Turning Around,"Eadweard Muybridge, ‘Descending Stairs and Tur..."
3,159897,Man Walking Around a Corner,The last remaining production of Le Prince's L...
4,16463,Roundhay Garden Scene,"The earliest surviving celluloid film, and bel..."


## Part 2 
Извлеките текстовые признаки у фильмов: tf idf, биграммы (может какие-то еще). Биграммы - статистика по биграммам, сколько какая биграмма встречается. Формулы для TF-IDF смотреть тут: https://en.wikipedia.org/wiki/Tf%E2%80%93idf 
    1. Используйте TF с double 0.5 normalization
    2. Используйте стандартный IDF: 
$$idf(t, D) = \frac{log N}{(1 + |d in D : t in d|)}$$

Получаем tf-idf для слов и биграмм:

In [95]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer(analyzer="word", ngram_range=(1, 2), input="content")

x = vectorizer.fit_transform(list(map(lambda x: x[1]['overview'], all_movies.items())))

sums = np.asarray(x.sum(axis=0))[0]
indices = sums.argsort()[::-1]
names = vectorizer.get_feature_names()
scored = [(names[index], sums[index]) for index in indices]

tfidfs = dict(scored)

In [96]:
list(map(lambda x: x[1]['overview'], all_movies.items()))

["Taisto Kasurinen is a Finnish coal miner whose father has just committed suicide and who is framed for a crime he did not commit. In jail, he starts to dream about leaving the country and starting a new life. He escapes from prison but things don't go as planned...",
 'An episode in the life of Nikander, a garbage man, involving the death of a co-worker, an affair and much more.',
 "It's Ted the Bellhop's first night on the job...and the hotel's very unusual guests are about to place him in some outrageous predicaments. It seems that this evening's room service is serving up one unbelievable happening after another.",
 'While racing to a boxing match, Frank, Mike, John and Rey get more than they bargained for. A wrong turn lands them directly in the path of Fallon, a vicious, wise-cracking drug lord. After accidentally witnessing Fallon murder a disloyal henchman, the four become his unwilling prey in a savage game of cat &amp; mouse as they are mercilessly stalked through the urban 

In [65]:
import nltk

In [100]:
import re

tmp = all_movies[2]['overview']
tmp = tmp.lower()
rgx = re.compile("([\w]*\w)")
# all_overviews_words = 
tmp_words = rgx.findall(tmp)

In [112]:
bigrams = list(nltk.ngrams(tmp_words, 2))
bigrams = list(set(map(lambda x: ' '.join(x), bigrams)))

In [113]:
[b in tfidfs for b in bigrams]

[True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True]

In [110]:
tfidfs['is a']

KeyError: 'is a'

In [75]:
from collections import Counter

In [78]:
c = Counter(all_overviews_words)

In [81]:
tmp = all_movies[2]['overview']

## Part 3 
Выведите подсчитанные признаки из пункта 2 для фильма с id 99114.

In [ ]:
# put your code here

## Part 4
Для каждого пользователя найдите топ жанров, которые нравятся пользователю (можно делать как на семинаре top 5, top 3 и top 2) и порекомендуйте таким пользователя фильмы близкие по текстовому описанию (см. пункт 2). Для оценки результатов пункта 3 используйте метрику HitRate и метрики ранжирования (N)DCG. Формулу для подсчета HitRate можно взять на семинаре,  (N)DCG: https://en.wikipedia.org/wiki/Discounted_cumulative_gain



In [ ]:
# put your code here